<a href="https://colab.research.google.com/github/Ayushman0Singh/BikeSharingDemandPrediction/blob/main/ModelTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing our data handling tools
import numpy as np
import pandas as pd
import numpy as math 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
#mounting drive
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Almabetter/Capstone Projects/Supervised ML-Regression /SeoulBikeData.csv'
df = pd.read_csv(file_path,encoding= 'unicode_escape')

Mounted at /content/drive


In [2]:
#Converting date to date-month-year
import datetime
df['Year'] = pd.DatetimeIndex(df['Date']).year   #getting year column
df['Month'] = pd.DatetimeIndex(df['Date']).month  #getting month column
df['Day'] = pd.DatetimeIndex(df['Date']).day  #getting Day column
df['Day_of_Week'] = pd.DatetimeIndex(df['Date']).weekday #Day of week 
df = df[df['Functioning Day'] == 'Yes']
df.sample(2)

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,Year,Month,Day,Day_of_Week
2706,23/03/2018,1611,18,9.7,36,3.6,948,-4.6,0.37,0.0,0.0,Spring,No Holiday,Yes,2018,3,23,4
7354,03/10/2018,1113,10,17.3,53,0.7,1406,7.6,1.83,0.0,0.0,Autumn,Holiday,Yes,2018,3,10,5


In [3]:
df_pr = df.copy()
df_pr.drop(labels = ['Date','Dew point temperature(°C)', 'Functioning Day' ] ,axis = 1,inplace = True)
df_pr = pd.get_dummies(df_pr, columns=["Seasons", "Holiday", 'Year', 'Month','Day','Day_of_Week', 'Hour' ])

In [4]:
df_pr.sample(2)

,Rented Bike Count,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons_Autumn,Seasons_Spring,...,Hour_14,Hour_15,Hour_16,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23
8659,1181,8.4,61,0.3,573,0.00,0.0,0.0,1,0,...,0,0,0,0,0,1,0,0,0,0
1595,156,-8.5,27,4.7,1894,1.73,0.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
features = list(df_pr.columns)[1:]
X = df_pr[features]
y = math.sqrt(df_pr['Rented Bike Count'])

In [6]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split( X,y , test_size = 0.2, random_state = 0) 
print(X_train.shape)
print(X_test.shape)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit_transform(X_train)
sc.transform(X_test)

(6772, 89)
(1693, 89)


array([[-1.5488159 , -1.27212732,  0.74002609, ..., -0.21265707,
        -0.20922027, -0.20690275],
       [ 0.85588137,  1.22372383, -0.22378972, ..., -0.21265707,
        -0.20922027, -0.20690275],
       [-0.338232  , -0.4891152 ,  2.28213139, ..., -0.21265707,
        -0.20922027, -0.20690275],
       ...,
       [ 0.78999926,  0.19602041, -0.80207921, ..., -0.21265707,
        -0.20922027, -0.20690275],
       [-0.66764258,  0.97903254, -0.22378972, ..., -0.21265707,
        -0.20922027, -0.20690275],
       [-1.25234637, -1.37000384,  0.93278925, ..., -0.21265707,
        -0.20922027, -0.20690275]])

In [7]:
#Reguarised linear Regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)
print('r2 for lr =', r2_score((y_test)**2, (y_pred)**2))

r2 for lr = 0.7287162043937694


In [9]:
#lasso/L1
from sklearn.linear_model import Lasso
Lasso = Lasso(alpha = 0.001, max_iter= 1000).fit(X_train,y_train)
y_pred_lasso = Lasso.predict(X_test)
print('r2 for lasso =', r2_score((y_test)**2, (y_pred_lasso)**2))

r2 for lasso = 0.7282643398969834


In [ ]:
#Ridge/L2
from sklearn.linear_model import Ridge
Ridge = Ridge(alpha = 6, max_iter= 1000).fit(X_train,y_train)
y_pred_ridge = Ridge.predict(X_test)
print('r2 for Ridge =', r2_score((y_test)**2, (y_pred_ridge)**2))

r2 for Ridge = 0.7272586766588252


In [10]:
#RMSLE
from sklearn.metrics import mean_squared_log_error
msle=mean_squared_log_error((y_test)**2, (y_pred_lasso)**2)
rmsle=np.sqrt(msle)
print('RMLSE for the data Lasso:',rmsle)

RMLSE for the data Lasso: 0.8276824961546995


In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [12]:
#Hyperparameter tuning for DT
hyperP_DT = {'max_depth':np.arange(8,100,2),'max_leaf_nodes': np.arange(100,500,10)}
DTregressor1 = GridSearchCV(DecisionTreeRegressor(random_state=88), hyperP_DT, cv=4)
DTregressor1.fit(X_train,y_train) 
y_pred_DTregressor1 = DTregressor1.predict(X_test)
DTregressor1.best_params_

{'max_depth': 22, 'max_leaf_nodes': 180}

In [ ]:
#Using the decision tree of the best parameters. 
print('r2 for DT1 =', r2_score((y_test)**2, (y_pred_DTregressor1)**2))
print('r2 on train set DT =', DTregressor1.score(X_train,y_train) )

#RMSLE
msle=mean_squared_log_error((y_test)**2, (y_pred_DTregressor1)**2)
rmsle=np.sqrt(msle)
print('RMLSE for the data DT1:',rmsle)
#Tree Properties
#print('Tree Depth',DTregressor1.get_depth())
# print('Tree Max Nodes', DTregressor1.get_n_leaves())

r2 for DT1 = 0.7743855281218548
r2 on train set DT = 0.9010135963050074
RMLSE for the data DT1: 0.5986789018876447


In [ ]:
# Trial Decision Tree Regressor
DTregressor = DecisionTreeRegressor(max_leaf_nodes = 400, random_state=88)
DTregressor.fit(X_train,y_train)
y_pred_DTregressor = DTregressor.predict(X_test)

In [ ]:
print('r2 for DT =', r2_score((y_test)**2, (y_pred_DTregressor)**2))
print('r2 on train set DT =', DTregressor.score(X_train,y_train) )
#M

#RMSLE
msle=mean_squared_log_error((y_test)**2, (y_pred_DTregressor)**2)
rmsle=np.sqrt(msle)
print('RMLSE for the data DT:',rmsle)
print('RMLSE for the data DT :',rmsle)
#Tree Properties
print('Tree Depth',DTregressor.get_depth())
print('Tree Max Nodes', DTregressor.get_n_leaves())

r2 for DT = 0.7569342826519919
r2 on train set DT = 0.9431212721545797
RMLSE for the data DT: 0.6108428588893801
RMLSE for the data DT : 0.6108428588893801
Tree Depth 26
Tree Max Nodes 400


In [ ]:
#RF
RFRegressor = RandomForestRegressor()
Hyperparam_RF = {'n_estimators':[50, 80 ,100], 'max_depth':np.arange(20,30,1)}
RFRegressor = GridSearchCV(RFRegressor, param_grid = Hyperparam_RF, cv=3)

# Fit the object to train dataset
RFRegressor.fit(X_train, y_train)
y_pred_RFregressor = RFRegressor.predict(X_test)
RFRegressor.best_params_

{'max_depth': 26, 'n_estimators': 100}

In [ ]:
y_pred_RFregressor_train = RFRegressor.predict(X_train)

In [ ]:
print('r2 for test-set rf =', r2_score((y_test)**2, (y_pred_RFregressor)**2))
print('r2 on train set rf =', r2_score((y_train)**2, (y_pred_RFregressor_train)**2))
#RMSLE
msle=mean_squared_log_error((y_test)**2, (y_pred_RFregressor)**2)
rmsle=np.sqrt(msle)
print('RMLSE for the data rf:',rmsle)


r2 for test-set rf = 0.8412918419260885
r2 on train set rf = 0.979106809680626
RMLSE for the data rf: 0.4835768246917128


In [15]:
#ADA-Boost
from sklearn.ensemble import AdaBoostRegressor
AdaRegressor = AdaBoostRegressor()
#Hyper-parameter tunning for AdaBoost
Hyperparam_AB = {'n_estimators':[100,150,200] ,'learning_rate': [0.1,1,10]}
AdaRegressor = GridSearchCV(AdaRegressor, param_grid= Hyperparam_AB, scoring='r2')

#fitting to train data
AdaRegressor.fit(X_train,y_train)
y_pred_AdaBoost = AdaRegressor.predict(X_test)
y_pred_AdaBoost_train = AdaRegressor.predict(X_train)
AdaRegressor.best_params_

{'learning_rate': 0.1, 'n_estimators': 200}

In [21]:
print('r2 for test-set ADA-Boost =', r2_score((y_test)**2, (y_pred_AdaBoost)**2))
print('r2 on train set ADA-Boost =', r2_score((y_train)**2, (y_pred_AdaBoost_train)**2))

r2 for test-set ADA-Boost = 0.5713739295749568
r2 on train set ADA-Boost = 0.5876363447259647


In [38]:
#GBM
from sklearn.ensemble import GradientBoostingRegressor
GBMregressor = GradientBoostingRegressor()
#Hyperparameter tunning
Hyperparam_GBM = {'learning_rate':[0.1], 'n_estimators':[100,500,1000]}
GBMregressor = GridSearchCV(GBMregressor,param_grid = Hyperparam_GBM )
#fitting to train data
GBMregressor.fit(X_train,y_train)
y_pred_GBMregressor = GBMregressor.predict(X_test)
y_pred_GBMregressor_train = GBMregressor.predict(X_train)
GBMregressor.best_params_


{'learning_rate': 0.1, 'n_estimators': 1000}

In [39]:
print('r2 for test-set GBM =', r2_score((y_test)**2, (y_pred_GBMregressor)**2))
print('r2 on train set GBM =', r2_score((y_train)**2, (y_pred_GBMregressor_train)**2))

r2 for test-set GBM = 0.8828475183978364
r2 on train set GBM = 0.9472524303669758
